<a href="https://colab.research.google.com/github/nikojim/llms/blob/main/gpt_rag_langchain_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
pip install langchain openai faiss-cpu langchain-community

In [2]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 1.7 MB/s eta 0:00:00


In [38]:
import os
import json
from glob import glob
from langchain_openai import OpenAIEmbeddings  # Correct package
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.docstore.document import Document
from google.colab import userdata


# Set API Key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')



In [23]:
# -------- Step 1: Load and Prepare Documents from Directory --------
def load_documents_from_directory(directory_path):
    documents = []

    json_files = glob(os.path.join(directory_path, "*.json"))

    for file_path in json_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                records = json.load(f)
                if isinstance(records, dict):
                    records = [records]
            except Exception as e:
                print(f"Failed to load {file_path}: {e}")
                continue

            for record in records:
                incidente = record.get("id", "")
                creador = record.get("owner", "")
                asunto = record.get("subject", "")
                estado = record.get("status", "")
                creado = record.get("created", "")
                ultima_modificacion = record.get("lastUpdated", "")
                transactions = record.get("transactions", [])
                transaction_texts = "\n".join(
                    f"[{t.get('fecha')}] {t.get('usuario')}: {t.get('texto', '')}"
                    for t in transactions
                )
                # Combine all into one document
                content = (
                    f"Incidente: {incidente}\n"
                    f"Creador: {creador}\n"
                    f"Estado: {estado}\n"
                    f"Asunto: {asunto}\n"
                    f"Creado: {creado}\n"
                    f"Última modificación: {ultima_modificacion}\n\n"
                    f"Transacciones:\n{transaction_texts}"
                )

                # add metadata
                metadata = {
                    "Incidente": incidente,
                    "Creador": creador,
                    "Asunto": asunto,
                    "Estado": estado,
                    "Creado": creado,
                }

                documents.append(Document(page_content=content, metadata=metadata))

    return documents


In [12]:
# -------- Step 2: Create FAISS DB with LangChain --------
def create_faiss_index(documents):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore



In [39]:
# -------- Step 3: RAG-style Query Function --------
def rag_query(query, vectorstore):
    relevant_docs = vectorstore.similarity_search(query, k=3)
    context = "\n\n".join(doc.page_content for doc in relevant_docs)

    llm = ChatOpenAI(model="gpt-4")

    messages = [
        SystemMessage(content=""""
            You are an assistant that responds for question-answering tasks. Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know.
            Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
            Answer in spanish.
            """),
        HumanMessage(content=f"Context:\n{context}\n\nQuestion: {query}")
    ]

    response = llm.invoke(messages)
    return response.content


In [40]:
# -------- Similarity searcha Faiss Query Function --------
def get_similar_documents(query: str, vectorstore: FAISS, k: int = 3):
    """
    Returns the top-k most similar documents from a FAISS vectorstore.

    Parameters:
    - query (str): The search query.
    - vectorstore (FAISS): The loaded FAISS vectorstore.
    - k (int): Number of top similar documents to return.

    Returns:
    - List[Document]: The most similar documents.
    """
    return vectorstore.similarity_search(query, k=k)

In [41]:
# -------- Filter by metadata --------
def filter_documents_by_metadata(vectorstore, filters: dict, k: int = 5):
    """
    Retrieve documents from a LangChain FAISS vectorstore by metadata filters.

    Parameters:
    - vectorstore: FAISS vectorstore (must be created with metadata)
    - filters (dict): e.g., {"creador": "lperna", "asunto": "Problemas en el mail"}
    - k (int): Number of documents to retrieve (optional)

    Returns:
    - List[Document]: Filtered documents
    """
    all_docs = vectorstore.similarity_search("", k=1000)  # empty query to get all docs
    filtered_docs = []

    for doc in all_docs:
        if all(doc.metadata.get(key) == value for key, value in filters.items()):
            filtered_docs.append(doc)
            if len(filtered_docs) >= k:
                break

    return filtered_docs

In [42]:
# -------- Main Execution --------
directory = "/content/sample_data/json-dataset"
docs = load_documents_from_directory(directory)



In [43]:
docs[1]


Document(metadata={'Incidente': '16059', 'Creador': 'lkarabogosian', 'Asunto': 'Solicitud 2 PC para equipo seguridad', 'Estado': 'resolved', 'Creado': '2025-01-02 19:09:08'}, page_content='Incidente: 16059\nCreador: lkarabogosian\nEstado: resolved\nAsunto: Solicitud 2 PC para equipo seguridad\nCreado: 2025-01-02 19:09:08\nÚltima modificación: 2025-01-14 15:18:00\n\nTransacciones:\n[2025-01-02 19:09:08] gguerrero@tilsor.com.uy: gguerrero@tilsor.com.uy\n[2025-01-02 19:09:08] gguerrero@tilsor.com.uy: Hola,  Ahora que est n con el cambio de PCs, quer a pedirles si 2 pueden ser  apartadas para ser utilizadas en el futuro por el equipo de seguridad  Lo ideal ser a tener:  - una de 16GB de RAM y la otra de 32GB de RAM - CPU ah  nos arreglamos pero de ser posible lo mejor de lo que les sobre - disco SSD para cada una para el SO - el SO le instalamos Linux nosotros despu s, as  que por eso no se  preocupen  Saludos, Guillermo         \n[2025-01-03 18:06:16] lperna: Buenas Guille,  No creo que t

In [44]:
#---------Load Faiss DB
if not docs:
    print("No documents found.")
else:
    vectorstore = create_faiss_index(docs)

In [45]:
#---------Test document search by context
query = "15440"
similar_docs = get_similar_documents(query, vectorstore, k=3)

for i, doc in enumerate(similar_docs, 1):
    print(f"--- Document {i} ---\n{doc.page_content}\n")

--- Document 1 ---
Incidente: 16036
Creador: lperna
Estado: resolved
Asunto: Mouse pad
Creado: 2024-12-31 12:02:28
Última modificación: 2025-01-03 17:25:53

Transacciones:
[2024-12-31 12:02:28] asanchez@tilsor.com.uy: asanchez@tilsor.com.uy
[2024-12-31 12:02:28] asanchez@tilsor.com.uy: Buen d a:  Estar a necesitando un mouse pad o similar.  Saludos. Agustin S.     

--- Document 2 ---
Incidente: 15101
Creador: lkarabogosian
Estado: resolved
Asunto: Testeo Nueva app Abitab
Creado: 2024-10-28 19:50:42
Última modificación: 2025-01-13 14:00:40

Transacciones:
[2024-10-28 19:50:42] rlopez: rlopez
[2024-10-28 19:50:42] rlopez: Buenas tardes, c mo hablamos con Fernanda,  en los pr ximos d as  estaremos liberando un aplicativo nuevo a Abitab, y previamente  necesitamos probar y validarlo su uso con los celulares , especialmente  por los navegadores espec ficos: Samsung, Safari .  Lo que hablamos con Fernanda hoy es que usar amos el acceso por internet  que se hizo para:  https://sirfe-rs.tilso

In [46]:
#---------Test document search by metadata
filters = {'Incidente': '16059'}
filtered_docs = filter_documents_by_metadata(vectorstore, filters, k=3)

for doc in filtered_docs:
    print("Metadata:", doc.metadata)
    print("Content:\n", doc.page_content)



Metadata: {'Incidente': '16059', 'Creador': 'lkarabogosian', 'Asunto': 'Solicitud 2 PC para equipo seguridad', 'Estado': 'resolved', 'Creado': '2025-01-02 19:09:08'}
Content:
 Incidente: 16059
Creador: lkarabogosian
Estado: resolved
Asunto: Solicitud 2 PC para equipo seguridad
Creado: 2025-01-02 19:09:08
Última modificación: 2025-01-14 15:18:00

Transacciones:
[2025-01-02 19:09:08] gguerrero@tilsor.com.uy: gguerrero@tilsor.com.uy
[2025-01-02 19:09:08] gguerrero@tilsor.com.uy: Hola,  Ahora que est n con el cambio de PCs, quer a pedirles si 2 pueden ser  apartadas para ser utilizadas en el futuro por el equipo de seguridad  Lo ideal ser a tener:  - una de 16GB de RAM y la otra de 32GB de RAM - CPU ah  nos arreglamos pero de ser posible lo mejor de lo que les sobre - disco SSD para cada una para el SO - el SO le instalamos Linux nosotros despu s, as  que por eso no se  preocupen  Saludos, Guillermo         
[2025-01-03 18:06:16] lperna: Buenas Guille,  No creo que tenga ninguna mother que

In [50]:
query = "problemas WAF"
answer = rag_query(query, vectorstore)
print(answer)

- Según el contexto proporcionado, durante el testeo de la nueva aplicación de Abitab, se solicitó la configuración del WAF (Firewall de Aplicaciones Web) en el ticket Mantis 17914 (Incidente 15101). 
- Este WAF fue configurado para permitir el acceso a ciertos recursos internos, específicamente la URL https://pki.tilsor.com.uy/pkiKeyGenerator/.
- Posteriormente, una vez completada la fase de prueba, el acceso externo a estos recursos se eliminó. Para esta acción, se creó un nuevo ticket Mantis (18218) solicitando la eliminación de la configuración del WAF que permitía el acceso externo al servidor pki.tilsor.com.uy. No se menciona ningún problema relacionado con el WAF en el contexto proporcionado.
